# Предсказание категории товара по описанию (день 2)
### Проверяем обученные алгоритмы на данных другого магазина.

Импортируем библиотеки:

In [16]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier


Подготавливаем обучающую выборку:

In [17]:
df = pd.read_csv('Dataset 10.21/products_dns_sorted.csv').dropna()
df_cat = pd.read_csv('Dataset 10.21/categories.csv').dropna()
df['category_id'].replace(df_cat.set_index('category_id').to_dict()['category_name'], inplace=True)
df.rename(columns = {'category_id': 'category'},  inplace = True)
df['description'] = df['description'].str.lower()

df = df.head(50000)

Создаем матрицу частот слов:

In [3]:
features = df['description']
answers = df['category']
cnt_vec = CountVectorizer()
features = cnt_vec.fit_transform(features)

Матрица частот ngam

In [19]:
ngr_cnt_vec = CountVectorizer(max_features = 1000, analyzer='char_wb', ngram_range=(2,3))
ngr_features = ngr_cnt_vec.fit_transform(df['description'])

Загружаем данные второго магазина

In [13]:
opp_df = pd.read_csv('Dataset 10.21/opponents_to_dns_products_250k.csv').dropna()
opp_df = opp_df.head(50000)
opp_df = opp_df[['opp_product_id', 'opp_product_description', 'category_id']]
opp_df['category_id'].replace(df_cat.set_index('category_id').to_dict()['category_name'], inplace=True)
opp_df.rename(columns = {'category_id': 'category'},  inplace = True)
opp_df['opp_product_description'] = opp_df['opp_product_description'].str.lower()
opp_df.head(5)

,opp_product_id,opp_product_description,category
0,8064001E6728A5A411E46B9A13D9EA47,фотокамера nikon d750 body,Зеркальные камеры Проф.
1,8064001E6728A5A411E46B9A13D9EA49,"портативная акустика sony srs-btv5w; белый""",Колонки портативные
2,8064001E6728A5A411E46B9A13D9EA49,"портативная акустика sony srs-btv5w; белый""",Колонки 2.0
3,8064001E6728A5A411E46B9A13D9EA4A,"монопод sony vct-amp1 для action cam""",Аксессуары для экшн-камер
4,8064001E6728A5A411E46B9A13D9EA4B,"портативная акустика sony srs-btv5b; черный""",Колонки портативные


Подготавливаем признаки для второго магазина

In [14]:
opp_features = opp_df['opp_product_description']
opp_answers = opp_df['category']
opp_features = cnt_vec.transform(opp_features)
ngr_opp_features = ngr_cnt_vec.transform(opp_df['opp_product_description'])

Наивный байесовский классификатор:

In [6]:
classifier = MultinomialNB().fit(features, answers)
print('train score:', "{0:.4f}".format(classifier.score(features, answers)))
print('test score:', "{0:.4f}".format(classifier.score(opp_features, opp_answers)))

train score: 0.8898
test score: 0.7758


Наивный байес ngram:

In [11]:
ngr_classifier = MultinomialNB().fit(ngr_features, answers)
print('train score:', "{0:.4f}".format(ngr_classifier.score(ngr_features, answers)))
print('test score:', "{0:.4f}".format(ngr_classifier.score(ngr_opp_features, opp_answers)))

train score: 0.8606
test score: 0.5582
